In [1]:
!pip install aitextgen
!pip install tensorflow

In [2]:
import json

# Open the dataset JSON
with open("dataSets/ingredient_and_instructions.json", "r") as dataSet:
    data = json.load(dataSet)

recipie = "" # The individual recipie
recipie_list = [] # Where we will store the recipie for training

for meal in data:
    # Get the recipie name and its instructions
    meal_info = data[meal]
    recipie += meal.replace("-", " ").title() + "\n\n"
    recipe_instructions = meal_info["instructions"]

    # Get the ingredients
    ingredients = meal_info["ingredient_sections"]

    for i in ingredients:
        # Retrieve ingredients and their details
        item = ""           # Dough
        qty = ""            # ½
        unit = ""           # cup
        ingredient = ""     # unsalted butter
        primary_unit = { "quantity": None, "display": None }

        # Some ingredients arent named, but are still listed as "1 cup" or similar
        # Additionally, if there is an ingredient name, it may already have a colon
        # in front of it, so we need to check for that before formatting
        if i["name"]:
            if i["name"][-1] != ":": # If there isnt a colon, add one
                item = i["name"] + ": \n"
            else: # Otherwise, just add the name
                item = i["name"] + " \n"
        else:
            item = ""

        recipie += item # Add the ingredient name to the recipie

        # Loop over each ingredient
        j = 0
        while j < len(i["ingredients"]):
            # i["ingredients"][0] may not exist, use try/except to prevent
            # any ListIndexErrors from accessing an index that doesn't exist
            try: primary_unit = i["ingredients"][j]["primary_unit"]
            except: primary_unit = { "quantity": None, "display": None } # Dummy object

            # Some recipies don't list an ingredient at all
            try: ingredient += i["ingredients"][j]["name"]
            except: ingredient += ""

            # If the try/except blocks above are successful, we can set the values
            if primary_unit["quantity"]: qty = primary_unit["quantity"] + " "
            if primary_unit["display"]: unit = primary_unit["display"] + " "

            # Construct the ingredient string
            recipie += qty + unit  + ingredient + "\n"
            ingredient = ""
            j += 1

        recipie += "\n"

    # Newline before adding instructions
    recipie += "\n"

    # Loop over the instructions and format them
    step = 1
    for text in recipe_instructions:
        instruction = text["display_text"]
        recipie += str(step) + ". " + instruction + "\n"
        step += 1

    # Add the recipie to the list
    recipie_list.append(recipie)

    # Free memory for the next recipie
    recipie = ""
    recipe_instructions = []
    ingredients = []
    item = ""
    qty = ""
    unit = ""
    ingredient = ""

# For testing purposes, we can output a random recipie
import random
print(recipie_list[random.randint(0, len(recipie_list) - 1)])

#print(recipie_list[0])


Meatball Call Of The Meatball

32 oz Great Value® Italian-Style Meatballs
3 cups marinara sauce
4 cups Marketside® Ciabatta Hamburger Buns
4 cups parmesan cheese


1. Preheat the oven to 375°F (190°C). Line a baking sheet with foil.
2. Place the meatballs on the prepared baking sheet. Bake for 30 minutes, or until the internal temperature reaches 165°F (75°C).
3. In a medium saucepan over medium heat, bring the marinara sauce to a simmer.
4. Place the meatballs and sauce on the ciabatta buns and top with Parmesan.
5. Enjoy!



In [3]:
#CSV File 1 Record per row
#Record: 1 string with ingredients + recipe


# [[recipe1], [recipe2], [recipe3]]

import csv

with open("recipe2.csv", 'w', encoding="utf-8") as csvfile:
    csvwriter = csv.writer(csvfile)
    for recipie in recipie_list:
        csvwriter.writerow([recipie])


--- MODEL TRANING ---

In [4]:
from aitextgen.TokenDataset import TokenDataset
from aitextgen.tokenizers import train_tokenizer
from aitextgen.utils import GPT2ConfigCPU
from aitextgen import aitextgen

tokenizer_file = "aitextgen.tokenizer.json"
config = GPT2ConfigCPU()
ai = aitextgen(tokenizer_file=tokenizer_file, to_gpu=True, config=config, tf_gpt2="355m")

C:\Users\flash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#Ai training disabled to prevent accidental overrides.


file_name = "recipe2.csv"
ai.save("aitextgen.model")
train_tokenizer(file_name)
data = TokenDataset(file_name, tokenizer_file=tokenizer_file, block_size=64)
ai.train(data, batch_size=1000, num_steps=64000, generate_every=1000, save_every=5000)
ai.generate(prompt="steak")

#Save the model
ai.save("aitextgen.model")

You are tokenizing a CSV file, but you did not set line_by_line=True. Please change if unintended.
100%|██████████| 9922/9922 [00:00<00:00, 13055.59it/s]
pytorch_model.bin already exists in /trained_model and will be overwritten!
Windows does not support multi-GPU training. Setting to 1 GPU.
C:\Users\flash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:151: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  rank_zero_deprecation(
C:\Users\flash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:96: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v

  0%|          | 0/64000 [00:00<?, ?it/s]

C:\Users\flash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pytorch_lightning\trainer\trainer.py:2281: LightningDeprecationWarning: `trainer.progress_bar_dict` is deprecated in v1.5 and will be removed in v1.7. Use `ProgressBarBase.get_metrics` instead.
  rank_zero_deprecation(


1,000 steps reached: generating sample texts.                                              
ut:                                                                                                
2pperkarow
 ch on alix
 chleeaspo˚na fro O inhehe
1. mix s thea powderzz asideonedin
 suilast witheas slicilgarardy.
2. mix srowil
2,000 steps reached: generating sample texts.                                              
ot Transferter.                                                                                    
4. ( bottom v stil intotred s Oneaspoonilight with lemon completely mo.
5. mix s cred singabavyilightedand.af lemonedandil whisk 30.
6. minutes f sach stickowls co
3,000 steps reached: generating sample texts.                                              
 ev vegetar fddinotk.                                                                              
2. mix soney sauce,il and mar with ve.
3. mix sicken,ese,ix,ilottomrowilorn with lemon completely mo.
4. mix co srowmver pariligh

In [8]:
# Load the trained model
ai = aitextgen(model_folder="trained_model", tokenizer_file="aitextgen.tokenizer.json")
ai.generate(prompt="pizza")

pizza dough on a lightly floured surface.
8. Roll the dough crosswise into a log about ⅛-inch (3 mm) thick.
9. Lightly roll the dough into a log, about 
